In [14]:
%run 'init.ipynb'

In [1]:
from backtesting.test import GOOG

,Open,High,Low,Close,Volume
2013-02-25,802.3,808.41,790.49,790.77,2303900
2013-02-26,795.0,795.95,784.40,790.13,2202500
2013-02-27,794.8,804.75,791.11,799.78,2026100
2013-02-28,801.1,806.99,801.03,801.20,2265800
2013-03-01,797.8,807.14,796.15,806.19,2175400


In [6]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()


In [7]:
from backtesting import Backtest

In [17]:
GOOG

,Open,High,Low,Close,Volume
2004-08-19,100.00,104.06,95.96,100.34,22351900
2004-08-20,101.01,109.08,100.50,108.31,11428600
2004-08-23,110.75,113.48,109.05,109.40,9137200
2004-08-24,111.24,111.60,103.57,104.87,7631300
2004-08-25,104.96,108.00,103.88,106.00,4598900
...,...,...,...,...,...
2013-02-25,802.30,808.41,790.49,790.77,2303900
2013-02-26,795.00,795.95,784.40,790.13,2202500
2013-02-27,794.80,804.75,791.11,799.78,2026100
2013-02-28,801.10,806.99,801.03,801.20,2265800


In [22]:
aapl.iloc[:,1:]

,Open,High,Low,Close,Volume
Day,,,,,
2021-08-02,146.3607,146.4606,146.3607,146.4606,6475
2021-08-02,146.4506,146.5404,146.4007,146.5005,20827
2021-08-02,146.5005,146.5305,146.4805,146.5305,4227
2021-08-02,146.5404,146.5604,146.5105,146.5205,3198
2021-08-02,146.5205,146.5604,146.5205,146.5604,3503
...,...,...,...,...,...
2021-08-12,148.9500,149.0000,148.9500,148.9715,680020
2021-08-12,148.9800,149.0300,148.9700,149.0100,476187
2021-08-12,149.0100,149.0500,149.0000,149.0450,404333


In [23]:
bt = Backtest(aapl.iloc[:,1:], SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats

<ipython-input-23-c65aa9b8f684>:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(aapl.iloc[:,1:], SmaCross, cash=10_000, commission=.002)


ValueError: cannot reindex from a duplicate axis